<a href="https://colab.research.google.com/github/anil-chhetri/Miscellaneous/blob/main/Azure%20Databricks/3%20Data%20Manuplication/DateTime_manipulation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://dlcdn.apache.org/spark/spark-3.2.1/spark-3.2.1-bin-hadoop3.2.tgz
!tar xf spark-3.2.1-bin-hadoop3.2.tgz
!pip install -q findspark
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.2.1-bin-hadoop3.2"
import findspark
findspark.init()
from pyspark.sql import SparkSession
spark = (SparkSession.builder
        .master("local")
        .appName("Colab")
        .config('spark.ui.port', '4050')
        .getOrCreate())

In [4]:
dummy = spark.createDataFrame([('x',)]).toDF('col')
dummy.show()

+---+
|col|
+---+
|  x|
+---+



In [6]:
from pyspark.sql import functions as f

In [7]:
help(f.current_date)

Help on function current_date in module pyspark.sql.functions:

current_date()
    Returns the current date at the start of query evaluation as a :class:`DateType` column.
    All calls of current_date within the same query return the same value.
    
    .. versionadded:: 1.5



In [9]:
dummy.select(f.current_date()).show()

+--------------+
|current_date()|
+--------------+
|    2022-05-14|
+--------------+



In [11]:
help(f.current_timestamp)

Help on function current_timestamp in module pyspark.sql.functions:

current_timestamp()
    Returns the current timestamp at the start of query evaluation as a :class:`TimestampType`
    column. All calls of current_timestamp within the same query return the same value.



In [15]:
dummy.select(f.current_timestamp()).show(truncate=False)

+-----------------------+
|current_timestamp()    |
+-----------------------+
|2022-05-14 07:29:12.755|
+-----------------------+



In [14]:
help(f.date_format)

Help on function date_format in module pyspark.sql.functions:

date_format(date, format)
    Converts a date/timestamp/string to a value of string in the format specified by the date
    format given by the second argument.
    
    A pattern could be for instance `dd.MM.yyyy` and could return a string like '18.03.1993'. All
    pattern letters of `datetime pattern`_. can be used.
    
    .. _datetime pattern: https://spark.apache.org/docs/latest/sql-ref-datetime-pattern.html
    
    .. versionadded:: 1.5.0
    
    Notes
    -----
    Whenever possible, use specialized functions like `year`.
    
    Examples
    --------
    >>> df = spark.createDataFrame([('2015-04-08',)], ['dt'])
    >>> df.select(date_format('dt', 'MM/dd/yyy').alias('date')).collect()
    [Row(date='04/08/2015')]



In [16]:
help(f.to_date)

Help on function to_date in module pyspark.sql.functions:

to_date(col, format=None)
    Converts a :class:`~pyspark.sql.Column` into :class:`pyspark.sql.types.DateType`
    using the optionally specified format. Specify formats according to `datetime pattern`_.
    By default, it follows casting rules to :class:`pyspark.sql.types.DateType` if the format
    is omitted. Equivalent to ``col.cast("date")``.
    
    .. _datetime pattern: https://spark.apache.org/docs/latest/sql-ref-datetime-pattern.html
    
    .. versionadded:: 2.2.0
    
    Examples
    --------
    >>> df = spark.createDataFrame([('1997-02-28 10:30:00',)], ['t'])
    >>> df.select(to_date(df.t).alias('date')).collect()
    [Row(date=datetime.date(1997, 2, 28))]
    
    >>> df = spark.createDataFrame([('1997-02-28 10:30:00',)], ['t'])
    >>> df.select(to_date(df.t, 'yyyy-MM-dd HH:mm:ss').alias('date')).collect()
    [Row(date=datetime.date(1997, 2, 28))]



In [24]:
dummy.select(f.to_date(f.lit('20220312'), 'yyyyMMdd').alias("date")).printSchema()

root
 |-- date: date (nullable = true)



In [26]:
dummy.select(f.to_timestamp(f.lit('20220312'), 'yyyyMMdd').alias("date")).show()

+-------------------+
|               date|
+-------------------+
|2022-03-12 00:00:00|
+-------------------+



Date Arithmetics

In [27]:
datetimes = [("2014-02-28", "2014-02-28 10:00:00.123"),
                     ("2016-02-29", "2016-02-29 08:08:08.999"),
                     ("2017-10-31", "2017-12-31 11:59:59.123"),
                     ("2019-11-30", "2019-08-31 00:00:00.000")
                ]

In [28]:
datetimeDf= spark.createDataFrame(datetimes, "date string, datetime string")

In [30]:
datetimeDf.show(truncate=False)

+----------+-----------------------+
|date      |datetime               |
+----------+-----------------------+
|2014-02-28|2014-02-28 10:00:00.123|
|2016-02-29|2016-02-29 08:08:08.999|
|2017-10-31|2017-12-31 11:59:59.123|
|2019-11-30|2019-08-31 00:00:00.000|
+----------+-----------------------+



In [31]:
help(f.date_add)

Help on function date_add in module pyspark.sql.functions:

date_add(start, days)
    Returns the date that is `days` days after `start`
    
    .. versionadded:: 1.5.0
    
    Examples
    --------
    >>> df = spark.createDataFrame([('2015-04-08',)], ['dt'])
    >>> df.select(date_add(df.dt, 1).alias('next_date')).collect()
    [Row(next_date=datetime.date(2015, 4, 9))]



In [32]:
## adding and subtracting 10 to both the dates.
(datetimeDf
  .withColumn("date_add", f.date_add('date', 10))
  .withColumn("datetimee_add", f.date_add('datetime', 10))
  .withColumn("date_sub", f.date_sub('date', 10))
  .withColumn('datetime_sub', f.date_sub('datetime', 10))
 ).show()

+----------+--------------------+----------+-------------+----------+------------+
|      date|            datetime|  date_add|datetimee_add|  date_sub|datetime_sub|
+----------+--------------------+----------+-------------+----------+------------+
|2014-02-28|2014-02-28 10:00:...|2014-03-10|   2014-03-10|2014-02-18|  2014-02-18|
|2016-02-29|2016-02-29 08:08:...|2016-03-10|   2016-03-10|2016-02-19|  2016-02-19|
|2017-10-31|2017-12-31 11:59:...|2017-11-10|   2018-01-10|2017-10-21|  2017-12-21|
|2019-11-30|2019-08-31 00:00:...|2019-12-10|   2019-09-10|2019-11-20|  2019-08-21|
+----------+--------------------+----------+-------------+----------+------------+



In [34]:
help(f.datediff)

Help on function datediff in module pyspark.sql.functions:

datediff(end, start)
    Returns the number of days from `start` to `end`.
    
    .. versionadded:: 1.5.0
    
    Examples
    --------
    >>> df = spark.createDataFrame([('2015-04-08','2015-05-10')], ['d1', 'd2'])
    >>> df.select(datediff(df.d2, df.d1).alias('diff')).collect()
    [Row(diff=32)]



In [36]:
# get the numbers of months between current_date and date values.
(
    datetimeDf
      .withColumn('monthsBetween', f.months_between(f.current_date(), 'date'))
      .withColumn('addMonths', f.add_months('date', 3))
).show()


+----------+--------------------+-------------+----------+
|      date|            datetime|monthsBetween| addMonths|
+----------+--------------------+-------------+----------+
|2014-02-28|2014-02-28 10:00:...|   98.5483871|2014-05-28|
|2016-02-29|2016-02-29 08:08:...|  74.51612903|2016-05-29|
|2017-10-31|2017-12-31 11:59:...|   54.4516129|2018-01-31|
|2019-11-30|2019-08-31 00:00:...|  29.48387097|2020-02-29|
+----------+--------------------+-------------+----------+



In [37]:
help(f.trunc)

Help on function trunc in module pyspark.sql.functions:

trunc(date, format)
    Returns date truncated to the unit specified by the format.
    
    .. versionadded:: 1.5.0
    
    Parameters
    ----------
    date : :class:`~pyspark.sql.Column` or str
    format : str
        'year', 'yyyy', 'yy' to truncate by year,
        or 'month', 'mon', 'mm' to truncate by month
        Other options are: 'week', 'quarter'
    
    Examples
    --------
    >>> df = spark.createDataFrame([('1997-02-28',)], ['d'])
    >>> df.select(trunc(df.d, 'year').alias('year')).collect()
    [Row(year=datetime.date(1997, 1, 1))]
    >>> df.select(trunc(df.d, 'mon').alias('month')).collect()
    [Row(month=datetime.date(1997, 2, 1))]



In [39]:
(
    datetimeDf
    .withColumn("trunc_date", f.trunc('date', "mm"))
    .withColumn('trunc_tme', f.trunc('datetime', 'yy'))
    
).show()

+----------+--------------------+----------+----------+
|      date|            datetime|trunc_date| trunc_tme|
+----------+--------------------+----------+----------+
|2014-02-28|2014-02-28 10:00:...|2014-02-01|2014-01-01|
|2016-02-29|2016-02-29 08:08:...|2016-02-01|2016-01-01|
|2017-10-31|2017-12-31 11:59:...|2017-10-01|2017-01-01|
|2019-11-30|2019-08-31 00:00:...|2019-11-01|2019-01-01|
+----------+--------------------+----------+----------+



In [41]:
help(f.date_trunc)

Help on function date_trunc in module pyspark.sql.functions:

date_trunc(format, timestamp)
    Returns timestamp truncated to the unit specified by the format.
    
    .. versionadded:: 2.3.0
    
    Parameters
    ----------
    format : str
        'year', 'yyyy', 'yy' to truncate by year,
        'month', 'mon', 'mm' to truncate by month,
        'day', 'dd' to truncate by day,
        Other options are:
        'microsecond', 'millisecond', 'second', 'minute', 'hour', 'week', 'quarter'
    timestamp : :class:`~pyspark.sql.Column` or str
    
    Examples
    --------
    >>> df = spark.createDataFrame([('1997-02-28 05:02:11',)], ['t'])
    >>> df.select(date_trunc('year', df.t).alias('year')).collect()
    [Row(year=datetime.datetime(1997, 1, 1, 0, 0))]
    >>> df.select(date_trunc('mon', df.t).alias('month')).collect()
    [Row(month=datetime.datetime(1997, 2, 1, 0, 0))]



In [46]:
(
    datetimeDf
    .withColumn('date_truc', f.date_trunc('MM', 'date'))
    .withColumn('time_truc', f.date_trunc('MM', 'datetime'))
    .withColumn('time_truc2', f.date_trunc('Hour', 'datetime'))


).show()

+----------+--------------------+-------------------+-------------------+-------------------+
|      date|            datetime|          date_truc|          time_truc|         time_truc2|
+----------+--------------------+-------------------+-------------------+-------------------+
|2014-02-28|2014-02-28 10:00:...|2014-02-01 00:00:00|2014-02-01 00:00:00|2014-02-28 10:00:00|
|2016-02-29|2016-02-29 08:08:...|2016-02-01 00:00:00|2016-02-01 00:00:00|2016-02-29 08:00:00|
|2017-10-31|2017-12-31 11:59:...|2017-10-01 00:00:00|2017-12-01 00:00:00|2017-12-31 11:00:00|
|2019-11-30|2019-08-31 00:00:...|2019-11-01 00:00:00|2019-08-01 00:00:00|2019-08-31 00:00:00|
+----------+--------------------+-------------------+-------------------+-------------------+



In [47]:
help(f.date_format)

Help on function date_format in module pyspark.sql.functions:

date_format(date, format)
    Converts a date/timestamp/string to a value of string in the format specified by the date
    format given by the second argument.
    
    A pattern could be for instance `dd.MM.yyyy` and could return a string like '18.03.1993'. All
    pattern letters of `datetime pattern`_. can be used.
    
    .. _datetime pattern: https://spark.apache.org/docs/latest/sql-ref-datetime-pattern.html
    
    .. versionadded:: 1.5.0
    
    Notes
    -----
    Whenever possible, use specialized functions like `year`.
    
    Examples
    --------
    >>> df = spark.createDataFrame([('2015-04-08',)], ['dt'])
    >>> df.select(date_format('dt', 'MM/dd/yyy').alias('date')).collect()
    [Row(date='04/08/2015')]

